# ULMFit

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#export
from exp.nb_12a import *

## Data
We load the data from 12a, instructions to create that file are there if you don't have it yet so go ahead and see.

In [3]:
path = untar_data(URLs.IMDB)

In [4]:
ll = pickle.load(open(path/'ll_lm.pkl', 'rb'))

In [5]:
bs, bptt = 128, 70
data = lm_databunchify(ll, bs, bptt)

In [6]:
vocab = ll.train.proc_x[1].vocab

## Finetuning the LM

Before tackling the classification task, we have to finetune our language model to the IMDB corpus.

We have pretrained a small model on [wikitext 103](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/) that you can download by uncommenting the following cell.

In [9]:
# ! wget http://files.fast.ai/models/wt103_tiny.tgz -P {path}
# ! tar xf {path}/wt103_tiny.tgz -C {path}

--2021-01-11 16:57:51--  http://files.fast.ai/models/wt103_tiny.tgz
Resolving files.fast.ai (files.fast.ai)... 104.26.3.19, 172.67.69.159, 104.26.2.19, ...
Connecting to files.fast.ai (files.fast.ai)|104.26.3.19|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.fast.ai/models/wt103_tiny.tgz [following]
--2021-01-11 16:57:51--  https://files.fast.ai/models/wt103_tiny.tgz
Connecting to files.fast.ai (files.fast.ai)|104.26.3.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75482451 (72M) [application/x-gtar-compressed]
Saving to: ‘/home/kienmn/.fastai/data/imdb/wt103_tiny.tgz’

wt103_tiny.tgz      100%[===================>]  71.99M  8.36MB/s    in 11s     

2021-01-11 16:58:03 (6.80 MB/s) - ‘/home/kienmn/.fastai/data/imdb/wt103_tiny.tgz’ saved [75482451/75482451]



In [7]:
dps = tensor([0.1, 0.15, 0.25, 0.02, 0.2]) * 0.5
tok_pad = None

In [8]:
emb_sz = 300
nh = 300
nl = 2

model = get_language_model(len(vocab), emb_sz, nh, nl, tok_pad, *dps)

In [9]:
old_wgts = torch.load(path/'pretrained'/'pretrained.pth')
old_vocab = pickle.load(open(path/'pretrained'/'vocab.pkl', 'rb'))

In our current vocabulary, it is very unlikely that the ids correspond to what is in the vocabulary used to train the pretrain model. The tokens are sorted by frequency (apart from the special tokens that are all first) so that order is specific to the corpus used. For instance, the word 'house' has different ids in the our current vocab and the pretrained one.

In [10]:
idx_house_new = vocab.index('house')
idx_house_old = old_vocab.index('house')
idx_house_new, idx_house_old

(346, 230)

We somehow need to match our pretrained weights to the new vocabulary. This is done on the embeddings and the decoder (since the weights between embeddings and decoders are tied) by putting the rows of the embedding matrix (or decoder bias) in the right order.

It may also happen that we have words that aren't in the pretrained vocab, in this case, we put the mean of the pretrained embedding weights/decoder bias.

In [11]:
house_wgt = old_wgts['0.emb.weight'][idx_house_old]
house_bias = old_wgts['1.decoder.bias'][idx_house_old]
house_wgt, house_bias

(tensor([ 0.5988, -0.4527,  0.8066, -0.2093,  0.5364, -0.2562, -0.1074, -0.2474,
         -0.0443,  0.4125,  0.3041, -0.5933,  0.2584, -1.1307, -0.0084,  0.4959,
         -1.1690,  0.1582, -0.2717,  0.6158, -0.4010,  0.0649,  0.5288, -0.6920,
         -0.2388,  0.9043, -0.6893, -0.3944,  0.6451,  0.1307,  0.8242, -0.3747,
         -0.0605, -0.3505,  0.0585,  0.4384,  0.1829, -1.3258,  0.3452,  0.1210,
          0.0926, -0.4796,  0.0763,  0.0323,  1.2245,  0.0214, -0.0185,  1.5935,
          0.7886, -0.3294, -1.6719, -0.2423, -0.3912, -0.0162,  0.8889, -0.0771,
          0.4504, -0.2190, -0.3146, -0.2552,  0.4464, -0.0984, -0.6560,  1.1554,
         -0.1786,  0.6311, -0.4426, -0.5250,  0.6636, -0.9760, -0.5825, -0.7320,
          0.3024, -0.2079,  0.4503, -0.2514,  0.1670,  0.8837, -0.2960, -0.2962,
          0.7779, -0.7776,  0.7375,  0.5516, -0.3084, -0.6680, -0.7311, -0.3653,
         -0.1539,  0.4493, -0.3127, -0.4654, -0.7501,  0.6402, -0.6743, -0.3092,
          0.8462, -0.5794, -

In [12]:
def match_embeds(old_wgts, old_vocab, new_vocab):
  wgts = old_wgts['0.emb.weight']
  bias = old_wgts['1.decoder.bias']
  wgts_m = wgts.mean(dim=0)
  bias_m = bias.mean()
  new_wgts = wgts.new_zeros(len(new_vocab), wgts.size(1))
  new_bias = bias.new_zeros(len(new_vocab))
  otoi = {v: k for k, v in enumerate(old_vocab)}
  for i, w in enumerate(new_vocab):
    if w in otoi:
      idx = otoi[w]
      new_wgts[i] = wgts[idx]
      new_bias[i] = bias[idx]
    else:
      new_wgts[i] = wgts_m
      new_bias[i] = bias_m
  old_wgts['0.emb.weight'] = new_wgts
  old_wgts['0.emb_dp.emb.weight'] = new_wgts
  old_wgts['1.decoder.weight'] = new_wgts
  old_wgts['1.decoder.bias'] = new_bias
  return old_wgts

In [13]:
wgts = match_embeds(old_wgts, old_vocab, vocab)

Now let's check that the word "*house*" was properly converted.

In [14]:
test_near(wgts['0.emb.weight'][idx_house_new], house_wgt)
test_near(wgts['1.decoder.bias'][idx_house_new], house_bias)

We can load the pretrained weights in our model before begining training.

In [15]:
model.load_state_dict(wgts)

<All keys matched successfully>

If we want to apply discriminative learning rates, we need to split our model in different layer groups. Let's have a look at our model.

In [16]:
model

SequentialRNN(
  (0): AWD_LSTM(
    (emb): Embedding(60003, 300)
    (emb_dp): EmbeddingDropout(
      (emb): Embedding(60003, 300)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(300, 300, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(300, 300, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (output_dp): RNNDropout()
    (decoder): Linear(in_features=300, out_features=60003, bias=True)
  )
)

Then we split by doing two groups for each rnn/corresponding dropout, then one last group that contains the embeddings/decoder. This is the one that needs to be trained the most as we may have new embeddings vectors.

In [17]:
def lm_splitter(m):
  groups = []
  for i in range(len(m[0].rnns)):
    groups.append(nn.Sequential(m[0].rnns[i], m[0].hidden_dps[i]))
  groups += [nn.Sequential(m[0].emb, m[0].emb_dp, m[0].input_dp, m[1])]
  return [list(o.parameters()) for o in groups]

First we train with the RNNs freezed.

In [18]:
for rnn in model[0].rnns:
  for p in rnn.parameters():
    p.requires_grad_(False)

In [19]:
device = torch.device('cuda', 0) if torch.cuda.is_available else torch.device('cpu')

cbs = [partial(AvgStatsCallback, accuracy_flat),
       partial(CudaCallback, device=device),
       Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, α=2., β=1.),
       ProgressCallback]

In [20]:
learn = Learner(model, data, cross_entropy_flat, opt_func=adam_opt(), cb_funcs=cbs, splitter=lm_splitter)

In [21]:
lr = 2e-2
cbsched = sched_1cycle([lr], pct_start=0.5, mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [ ]:
learn.fit(1, cbs=cbsched)

The the whole model with discriminative learning rates.

In [22]:
for rnn in model[0].rnns:
  for p in rnn.parameters():
    p.requires_grad_(True)

In [23]:
lr = 2e-3
cbsched = sched_1cycle([lr/2., lr/2., lr], pct_start=0.5, mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [ ]:
learn.fit(10, cbs=cbsched)

We only need to save the encoder (first part of the model) for the classification, as well as the vocabulary used (we will need to use the same in the classification task).

In [ ]:
torch.save(learn.model[0].state_dict(), path/'finetuned_enc.pth')

In [24]:
pickle.dump(vocab, open(path/'vocab_lm.pkl', 'wb'))

In [ ]:
torch.save(learn.model.state_dict(), path/'funetuned.pth')

## Classifier

In [25]:
vocab = pickle.load(open(path/'vocab_lm.pkl', 'rb'))
proc_tok = TokenizeProcessor()
proc_num = NumericalizeProcessor(vocab=vocab)
proc_cat = CategoryProcessor()

In [26]:
il = TextList.from_files(path, include=['train', 'test'])
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='test'))
ll = label_by_func(sd, parent_labeler, proc_x=[proc_tok, proc_num], proc_y=proc_cat)

In [27]:
pickle.dump(ll, open(path/'ll_class.pkl', 'wb'))

In [28]:
ll = pickle.load(open(path/'ll_class.pkl', 'rb'))
vocab = pickle.load(open(path/'vocab_lm.pkl', 'rb'))

In [29]:
bs, bptt = 64, 70
data = clas_databunchify(ll, bs)

### Ignore padding
We will those two utility functions from PyTorch to ignore the padding in the inputs.

In [30]:
#export
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

Let's see how this works: first we grab a batch of the training set.

In [31]:
x, y = next(iter(data.train_dl))

In [32]:
x.size()

torch.Size([64, 3311])

We need to pass to the utility functions the lengths of our sentences because it's applied after the embedding, so we can't see the padding anymore.

In [33]:
lengths = x.size(1) - (x == 1).sum(1)
lengths[:5]

tensor([3311, 1577, 1481, 1443, 1390])

In [34]:
tst_emb = nn.Embedding(len(vocab), 300)

In [35]:
tst_emb(x).shape

torch.Size([64, 3311, 300])

In [36]:
128 * 70

8960

We create a `PackedSequence` object that contains all of our unpadded sequences

In [37]:
packed = pack_padded_sequence(tst_emb(x), lengths, batch_first=True)

In [38]:
packed

PackedSequence(data=tensor([[-0.5455,  0.7309,  0.1489,  ..., -0.1312,  0.9794,  0.2039],
        [-0.5455,  0.7309,  0.1489,  ..., -0.1312,  0.9794,  0.2039],
        [-0.5455,  0.7309,  0.1489,  ..., -0.1312,  0.9794,  0.2039],
        ...,
        [ 0.1935, -0.3665,  1.0036,  ..., -0.3770,  0.5414, -0.8141],
        [-0.5708, -1.0967, -0.8068,  ..., -1.1290,  1.3198, -0.4017],
        [-1.1747, -0.7042,  0.8974,  ..., -0.2568, -0.8326,  1.3060]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([64, 64, 64,  ...,  1,  1,  1]), sorted_indices=None, unsorted_indices=None)

In [39]:
packed.data.shape

torch.Size([76136, 300])

In [40]:
len(packed.batch_sizes)

3311

In [41]:
8960 // 70

128

This object can be passed to any RNN directly while retaining the speed of CuDNN.

In [42]:
tst = nn.LSTM(300, 300, 2)

In [43]:
y, h = tst(packed)

Then we can unpad it with the following function for other modules:

In [44]:
unpack = pad_packed_sequence(y, batch_first=True)

In [45]:
unpack[0].shape

torch.Size([64, 3311, 300])

In [46]:
unpack[1]

tensor([3311, 1577, 1481, 1443, 1390, 1369, 1341, 1336, 1323, 1289, 1284, 1279,
        1270, 1267, 1267, 1260, 1260, 1252, 1247, 1244, 1241, 1235, 1214, 1211,
        1200, 1191, 1186, 1183, 1180, 1160, 1160, 1151, 1143, 1139, 1137, 1101,
        1077, 1073, 1063, 1052, 1049, 1046, 1043, 1042, 1042, 1039, 1033, 1030,
        1030, 1027, 1026, 1026, 1020, 1020, 1016, 1016, 1014, 1013, 1012, 1011,
        1006, 1001,  994,  993])

We need to change our model a little bit to use this.

In [47]:
#export
class AWD_LSTM1(nn.Module):
  "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182."
  initrange=0.1

  def __init__(self,
               vocab_sz,
               emb_sz,
               n_hid,
               n_layers,
               pad_token,
               hidden_p=0.2,
               input_p=0.6,
               embed_p=0.1,
               weight_p=0.5):
    super().__init__()
    self.bs = 1
    self.emb_sz = emb_sz
    self.n_hid = n_hid
    self.n_layers = n_layers
    self.pad_token = pad_token
    self.emb = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
    self.emb_dp = EmbeddingDropout(self.emb, embed_p)
    self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid,
                         n_hid if l != n_layers - 1 else emb_sz,
                         1,
                         batch_first=True)
                 for l in range(n_layers)]
    self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns])
    self.emb.weight.data.uniform_(-self.initrange, self.initrange)
    self.input_dp = RNNDropout(input_p)
    self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

  def forward(self, input):
    bs, sl = input.size()
    mask = (input == self.pad_token)
    lengths = sl - mask.long().sum(1)
    n_empty = (lengths == 0).sum()
    if n_empty > 0:
      input = input[:-n_empty]
      lengths = lengths[:-n_empty]
      self.hidden = [(h[0][:,:input.size(0)], h[1][:,:input.size(0)]) for h in self.hidden]
    raw_output = self.input_dp(self.emb_dp(input))
    new_hidden,raw_outputs,outputs = [],[],[]
    for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
      raw_output = pack_padded_sequence(raw_output, lengths, batch_first=True)
      raw_output, new_h = rnn(raw_output, self.hidden[l])
      raw_output = pad_packed_sequence(raw_output, batch_first=True)[0]
      raw_outputs.append(raw_output)
      if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
      outputs.append(raw_output)
      new_hidden.append(new_h)
    self.hidden = to_detach(new_hidden)
    return raw_outputs, outputs, mask

  def _one_hidden(self, l):
    "Return one hidden state."
    nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
    return next(self.parameters()).new(1, self.bs, nh).zero_()

  def reset(self):
    "Reset the hidden states."
    self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

### Concat pooling
We will use three things for the classification head of the model: the last hidden state, the average of all the hidden states and the maximum of all the hidden states. The trick is just to, once again, ignore the padding in the last element/average/maximum.

In [53]:
class Pooling(nn.Module):
  def forward(self, input):
    raw_outputs, outputs, mask = input
    output = outputs[-1]
    lengths = output.size(1) - mask.long().sum(dim=1)
    avg_pool = output.masked_fill(mask[:, :, None], 0).sum(dim=1)
    avg_pool.div_(lengths.type(avg_pool.dtype)[:, None])
    max_pool = output.masked_fill(mask[:, :, None], -float('inf')).max(dim=1)[0]
    x = torch.cat([output[torch.arange(0, output.size(0)), lengths - 1], max_pool, avg_pool], 1) # Concat pooling.
    return output, x

In [54]:
emb_sz = 300
nh = 300
nl = 2
tok_pad = vocab.index(PAD)

In [55]:
enc = AWD_LSTM1(len(vocab), emb_sz, n_hid=nh, n_layers=nl, pad_token=tok_pad)
pool = Pooling()
enc.bs = bs
enc.reset()

In [56]:
x, y = next(iter(data.train_dl))
output, c = pool(enc(x))

We can check we have padding with 1s at the end of each text (except the first which is the longest).

In [57]:
x

tensor([[    2,     7,  1165,  ..., 15280,    24,     3],
        [    2,     7,   148,  ...,     1,     1,     1],
        [    2,     7,  1330,  ...,     1,     1,     1],
        ...,
        [    2,     7,    63,  ...,     1,     1,     1],
        [    2,    21,     7,  ...,     1,     1,     1],
        [    2,     7,    64,  ...,     1,     1,     1]])

PyTorch puts 0s everywhere we had padding in the `output` when unpacking.

In [58]:
test_near((output.sum(dim=2) == 0).float(), (x == tok_pad).float())

So the last hidden state isn't the last element of `output`. Let's check we got everything right.

In [59]:
for i in range(bs):
  length = x.size(1) - (x[i] == 1).long().sum()
  out_unpad = output[i, : length]
  test_near(out_unpad[-1], c[i, :300])
  test_near(out_unpad.max(0)[0], c[i, 300: 600])
  test_near(out_unpad.mean(0), c[i, 600:])

Our pooling layer properly ignored the padding, so now let's group it with a classifier.

In [60]:
def bn_drop_lin(n_in, n_out, bn=True, p=0., actn=None):
  layers = [nn.BatchNorm1d(n_in)] if bn else []
  if p != 0:
    layers.append(nn.Dropout(p))
  layers.append(nn.Linear(n_in, n_out))
  if actn is not None:
    layers.append(actn)
  return layers

In [70]:
class PoolingLinearClassifier(nn.Module):
  "Create a linear classifier with pooling."
  
  def __init__(self, layers, drops):
    super().__init__()
    mod_layers = []
    activs = [nn.ReLU(inplace=True)] * (len(layers) - 2) + [None]
    for n_in, n_out, p, actn in zip(layers[:-1], layers[1:], drops, activs):
      mod_layers += bn_drop_lin(n_in, n_out, p=p, actn=actn)
    self.layers = nn.Sequential(*mod_layers)
    
  def forward(self, input):
    raw_outputs, outputs, mask = input
    output = outputs[-1]
    lengths = output.size(1) - mask.long().sum(dim=1)
    avg_pool = output.masked_fill(mask[:, :, None], 0).sum(dim=1)
    avg_pool.div_(lengths.type(avg_pool.dtype)[:, None])
    max_pool = output.masked_fill(mask[:, :, None], -float('inf')).max(dim=1)[0]
    x = torch.cat([output[torch.arange(0, output.size(0)), lengths - 1], max_pool, avg_pool], 1) # Concat pooling.
    x = self.layers(x)
    return x

Then we just have to feed our texts to those two blocks, (but we can't give them all at once to the AWD_LSTM or we might get OOM error: we'll go for chunks of bptt length to regularly detach the history of our hidden states.)

In [71]:
def pad_tensor(t, bs, val=0.):
  if t.size(0) < bs:
    return torch.cat([t, val + t.new_zeros(bs - t.size(0), *t.shape[1:])])
  return t

In [72]:
class SentenceEncoder(nn.Module):
  def __init__(self, module, bptt, pad_idx=1):
    super().__init__()
    self.bptt = bptt
    self.module = module
    self.pad_idx = pad_idx
    
  def concat(self, arrs, bs):
    return [torch.cat([pad_tensor(l[si], bs) for l in arrs], dim=1) for si in range(len(arrs[0]))]
  
  def forward(self, input):
    bs, sl = input.size()
    self.module.bs = bs
    self.module.reset()
    raw_outputs, outputs, masks = [], [], []
    for i in range(0, sl, self.bptt):
      r, o, m = self.module(input[:, i: min(i + self.bptt, sl)])
      masks.append(pad_tensor(m, bs, 1))
      raw_outputs.append(r)
      outputs.append(o)
    return self.concat(raw_outputs, bs), self.concat(outputs, bs), torch.cat(masks, dim=1)

In [73]:
def get_text_classifier(vocab_sz,
                        emb_sz,
                        n_hid,
                        n_layers,
                        n_out,
                        pad_token,
                        bptt,
                        output_p=0.4,
                        hidden_p=0.2,
                        input_p=0.6,
                        embed_p=0.1,
                        weight_p=0.5,
                        layers=None,
                        drops=None):
  "To create a full AWD-LSTM"
  rnn_enc = AWD_LSTM1(vocab_sz,
                      emb_sz,
                      n_hid=n_hid,
                      n_layers=n_layers,
                      pad_token=pad_token,
                      hidden_p=hidden_p,
                      input_p=input_p,
                      embed_p=embed_p,
                      weight_p=weight_p)
  enc = SentenceEncoder(rnn_enc, bptt)
  if layers is None:
    layers = [50]
  if drops is None:
    drops = [0.1] * len(layers)
  layers = [3 * emb_sz] + layers + [n_out]
  drops = [output_p] + drops
  return SequentialRNN(enc, PoolingLinearClassifier(layers, drops))

In [74]:
emb_sz = 300
nh = 300
nl = 2
dps = tensor([0.4, 0.3, 0.4, 0.05, 0.5]) * 0.25
model = get_text_classifier(len(vocab), emb_sz, nh, nl, 2, 1, bptt, *dps)

In [83]:
model

SequentialRNN(
  (0): SentenceEncoder(
    (module): AWD_LSTM1(
      (emb): Embedding(60003, 300, padding_idx=1)
      (emb_dp): EmbeddingDropout(
        (emb): Embedding(60003, 300, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(300, 300, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(300, 300, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): BatchNorm1d(900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.10000000149011612, inplace=False)
      (2): Linear(in_features=900, out_features=50, bias=True)
      (3): ReLU(inplace=True)
      (4): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Dropout(p=0.1, inplace=False)
      (6): 

### Training
We load our pretrained encoder and freeze it.

In [75]:
def class_splitter(m):
  enc = m[0].module
  groups = [nn.Sequential(enc.emb, enc.emb_dp, enc.input_dp)]
  for i in range(len(enc.rnns)):
    groups.append(nn.Sequential(enc.rnns[i], enc.hidden_dps[i]))
  groups.append(m[1])
  return [list(o.parameters()) for o in groups]

In [77]:
for p in model[0].parameters():
  p.requires_grad_(False)

In [78]:
device = torch.device('cuda', 0) if torch.cuda.is_available else torch.device('cpu')

cbs = [partial(AvgStatsCallback, accuracy_flat),
       partial(CudaCallback, device=device),
       Recorder,
       partial(GradientClipping, clip=0.1),
       ProgressCallback]

In [ ]:
model[0].module.load_state_dict(torch.load(path/'finetuned_enc.pth'))

In [79]:
learn = Learner(model, data, F.cross_entropy, opt_func=adam_opt(), cb_funcs=cbs, splitter=class_splitter)

In [80]:
lr = 1e-2
cbsched = sched_1cycle([lr], mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [ ]:
learn.fit(1, cbs=cbsched)

In [81]:
for p in model[0].module.rnns[-1].parameters():
  p.requires_grad_(True)

In [82]:
lr = 5e-3
cbsched = sched_1cycle([lr/2., lr/2., lr/2., lr], mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [ ]:
learn.fit(1, cbs=cbsched)

In [85]:
for p in model[0].parameters():
  p.requires_grad_(True)

In [86]:
lr = 1e-3
cbsched = sched_1cycle([lr/8., lr/4., lr/2., lr], mom_start=0.8, mom_mid=0.7, mom_end=0.8)

In [ ]:
learn.fit(2, cbs=cbsched)

In [87]:
x, y = next(iter(data.valid_dl))

Predicting on the padded batch or on the individual unpadded samples give the same results.

In [ ]:
pred_batch = learn.model.eval()(x.cuda())

In [ ]:
pred_ind = []
for inp in x:
  length = x.size(1) - (inp == 1).long().sum()
  inp = inp[: length]
  pred_ind.append(learn.model.eval()(inp[None].cuda()))

In [ ]:
assert near(pred_batch, torch.cat(pred_ind))